In [4]:
import sys
import shutil
import os
from glob import glob
from types import ModuleType

# --- ZONE DE PATCH (POUR PYTHON 3.13) ---
# Ce bloc répare l'erreur 'imghdr' en créant un faux module à la volée
if 'imghdr' not in sys.modules:
    print("🔧 Application du patch pour Python 3.13...")
    fake_imghdr = ModuleType('imghdr')
    # On définit une fonction 'what' bidon qui renvoie toujours None ou 'jpeg'
    # (La librairie l'utilise juste pour vérifier si le fichier est une image)
    def what(filename, h=None):
        return 'jpeg'
    fake_imghdr.what = what
    sys.modules['imghdr'] = fake_imghdr
# ----------------------------------------

# Maintenant on peut importer sans erreur
from bing_image_downloader import downloader

def deplacer_et_repartir(source_folder, dest_root, class_name, ratio_train=0.8):
    """Déplace les images téléchargées vers train/val"""
    
    # Trouver toutes les images téléchargées
    extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp']
    images = []
    for ext in extensions:
        images.extend(glob(os.path.join(source_folder, ext)))
    
    # Calcul du point de coupure
    limit_train = int(len(images) * ratio_train)
    
    # Dossiers de destination
    dir_train = os.path.join(dest_root, 'train', class_name)
    dir_val = os.path.join(dest_root, 'val', class_name)
    os.makedirs(dir_train, exist_ok=True)
    os.makedirs(dir_val, exist_ok=True)
    
    print(f"   📦 Tri : {limit_train} images -> Train | {len(images) - limit_train} images -> Val")
    
    # Déplacement
    for i, img_path in enumerate(images):
        filename = os.path.basename(img_path)
        nouv_nom = f"{class_name}_{i}.jpg" # On renomme proprement
        
        if i < limit_train:
            shutil.move(img_path, os.path.join(dir_train, nouv_nom))
        else:
            shutil.move(img_path, os.path.join(dir_val, nouv_nom))

def generer_dataset(liste_personnages, n_images=80):
    dataset_dir = "dataset"
    temp_dir = "temp_downloads" # Dossier temporaire pour le téléchargement brut
    
    print(f"🚀 Démarrage (Mode Bing - Robuste)")

    for perso in liste_personnages:
        nom_classe = perso.replace(" ", "_").lower()
        
        # 1. Cas "MOI"
        if nom_classe == "romain":
            os.makedirs(os.path.join(dataset_dir, "train", "romain"), exist_ok=True)
            os.makedirs(os.path.join(dataset_dir, "val", "romain"), exist_ok=True)
            print(f"\n👤 Classe 'MOI' préparée (Dossiers vides).")
            continue
            
        # 2. Téléchargement (Bing)
        print(f"\n🌐 Téléchargement de {perso}...")
        query = f"{perso} face"
        print(query)
        
        try:
            # On télécharge dans un dossier temporaire
            downloader.download(
                query, 
                limit=n_images, 
                output_dir=temp_dir, 
                adult_filter_off=True, 
                force_replace=False, 
                timeout=5, 
                verbose=False
            )
            
            # Le dossier créé par Bing est souvent le nom de la query
            # On cherche le dossier qui vient d'être créé
            dossier_telecharge = os.path.join(temp_dir, query)
            
            # 3. Répartition Train / Val
            if os.path.exists(dossier_telecharge):
                deplacer_et_repartir(dossier_telecharge, dataset_dir, nom_classe)
            else:
                print("❌ Erreur : Dossier introuvable, Bing a peut-être échoué.")
                
        except Exception as e:
            print(f"⚠️ Petite erreur sur {perso}: {e}")

    # Nettoyage du dossier temporaire
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
        print("\n🧹 Nettoyage des fichiers temporaires terminé.")

    print(f"\n✨ Terminé ! Ton dataset est prêt dans '{dataset_dir}'.")


MA_LISTE = [
    "Romain",              
    "Ryan Gosling",       
    "Gollum",             
    "Voldemort",           
    "Margot Robbie" 
]

# --- LANCEMENT ---
if __name__ == "__main__":
    generer_dataset(MA_LISTE, n_images=80)


🚀 Démarrage (Mode Bing - Robuste)

👤 Classe 'MOI' préparée (Dossiers vides).

🌐 Téléchargement de Ryan Gosling...
Ryan Gosling face
[%] Downloading Images to c:\Users\romai\Desktop\Centrale lyon\MOD et MOM\Intro apprentissage profond et IA\TD\mod_4_6-td2_cnn\temp_downloads\Ryan Gosling face
[!] Issue getting: https://ryans.world/wp-content/uploads/2019/12/ryan-bio-photo-827x1024.png
[!] Error:: HTTP Error 403: Forbidden
[!] Issue getting: https://mms.businesswire.com/media/20231115932518/en/1944945/23/Ryan_Logo1A_(003).jpg
[!] Error:: The read operation timed out
[!] Issue getting: https://wl-brightside.cf.tsp.li/resize/1200x630/jpg/2fd/a5c/d5a2cf5c1b844366cf96e3aa7d.jpg
[!] Error:: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: Basic Constraints of CA cert not marked critical (_ssl.c:1020)>


[%] Done. Downloaded 80 images.
   📦 Tri : 57 images -> Train | 15 images -> Val

🌐 Téléchargement de Gollum...
Gollum face
[%] Downloading Images to c:\Users\romai\De